In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#본인 구글드라이브에 EmotionLines_friends_annotation.tar.gz(캐글) 를 넣어놈
!gzip -d /content/drive/MyDrive/EmotionLines_friends_annotation.tar.gz

gzip: /content/drive/MyDrive/EmotionLines_friends_annotation.tar.gz: No such file or directory


In [ ]:
!tar xvf /content/drive/MyDrive/EmotionLines_friends_annotation.tar

EmotionLines/Friends/friends_train.json
EmotionLines/Friends/friends_dev.json
EmotionLines/Friends/friends_test.json
EmotionLines/README


In [ ]:
import tensorflow as tf
import torch


import pandas as pd
import numpy as np
import random
import time
import datetime
import pickle
import os
import json
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import keras
import sys
import pickle
import re
import time
import warnings

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, cross_val_score,learning_curve
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix


In [ ]:
def jsonToDf(file_name):
  with open(file_name, encoding = 'utf-8', mode = 'r') as file:
    json_array = json.load(file)
  
  result = pd.DataFrame.from_dict(json_array[0])

  is_first = True
  for array in json_array:
    if is_first:
      is_first = False
      continue
    
    temp_df = pd.DataFrame.from_dict(array)
    result = result.append(temp_df, ignore_index = True)

  return result

In [ ]:
train = jsonToDf('EmotionLines/Friends/friends_train.json')
dev = jsonToDf('EmotionLines/Friends/friends_dev.json')
test= jsonToDf('EmotionLines/Friends/friends_test.json')
train = train.append(dev, ignore_index = True)



In [ ]:
leader_test = pd.read_csv('/content/drive/MyDrive/en_data.csv', encoding = 'unicode_escape')
leader_test.rename(columns = {test.columns[0] : 'id'}, inplace = True)

In [ ]:
a=train.emotion.drop_duplicates()

In [ ]:
emotion=a.to_list()

In [ ]:
emotion

['neutral',
 'surprise',
 'fear',
 'non-neutral',
 'joy',
 'sadness',
 'anger',
 'disgust']

In [ ]:
train = train[['utterance','emotion']]
test  = test[['utterance','emotion']]

In [ ]:
def str_to_ind(train):
  ret=[]
  for emo in train.emotion:
    i=0
    for tmp in emotion:
      if tmp == emo:
        ret.append(i)
      i+=1
  return pd.DataFrame(ret)

In [ ]:
def ind_to_str(indlist):
  ret=[]
  for ind in indlist:
    ret.append(emotion[int(ind)])
  return pd.DataFrame(ret)

In [ ]:
b=str_to_ind(train)

In [ ]:
train_data=pd.concat([train, b], axis=1, ignore_index=True)
test_data=pd.concat([test, b], axis=1, ignore_index=True)
train_data=train_data.dropna()
test_data=test_data.dropna()

In [ ]:
train_data.columns = ['utterance','emotion','cls']
test_data.columns = ['utterance','emotion','cls']

In [ ]:
train_X=train_data ['utterance']
test_X=test_data ['utterance']

In [ ]:
train_y=train_data['cls']
test_y=test_data['cls']

### 토큰 전처리

In [ ]:
def tokenize_data(text:str) -> list:
    #tokenizer = RegexpTokenizer("[\w']+")
    english_stops = set(stopwords.words('english'))
    words=word_tokenize(text)

    return [word for word in words if word not in english_stops]
    #return [word for word in words ]

### 모델 생성 및 결과 도출

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=tokenize_data)),  # bow로 word2vec
    ('tfidf', TfidfTransformer(use_idf=True)),  # weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # 나이브베이즈 분류기
])

In [ ]:
%time model_nb = pipeline.fit(train_X, train_y)

CPU times: user 1.46 s, sys: 885 µs, total: 1.46 s
Wall time: 1.46 s


In [ ]:
prediction=model_nb.predict(test_X)

### 나이브 베이즈 모델 결과

In [ ]:
print (classification_report(test_y, prediction))

              precision    recall  f1-score   support

           0       0.45      0.94      0.60      1240
           1       0.09      0.02      0.03       309
           2       0.00      0.00      0.00        51
           3       0.18      0.03      0.05       521
           4       0.11      0.00      0.01       350
           5       0.00      0.00      0.00        93
           6       0.00      0.00      0.00       141
           7       0.00      0.00      0.00        59

    accuracy                           0.43      2764
   macro avg       0.10      0.12      0.09      2764
weighted avg       0.26      0.43      0.28      2764



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pipeline_randomforest=Pipeline([
            ('bow', CountVectorizer(tokenizer=tokenize_data, ngram_range=(1,2))),
            ('tfidf', TfidfTransformer(use_idf=False)),
            ('classifier', RandomForestClassifier(n_estimators=100, oob_score=True))
            ])

In [ ]:
%time model_randomforest = pipeline_randomforest.fit(train_X, train_y)

CPU times: user 55.7 s, sys: 122 ms, total: 55.8 s
Wall time: 55.8 s


In [ ]:
prediction_randomforest=model_randomforest.predict(test_X)

### 랜덤포레스트 모델 결과

In [ ]:
print (classification_report(test_y, prediction_randomforest))

              precision    recall  f1-score   support

           0       0.44      0.68      0.54      1240
           1       0.11      0.09      0.10       309
           2       0.00      0.00      0.00        51
           3       0.19      0.09      0.12       521
           4       0.11      0.09      0.10       350
           5       0.00      0.00      0.00        93
           6       0.04      0.01      0.01       141
           7       0.00      0.00      0.00        59

    accuracy                           0.35      2764
   macro avg       0.11      0.12      0.11      2764
weighted avg       0.26      0.35      0.29      2764



In [ ]:
pipeline_gdboost=Pipeline([
            ('bow', CountVectorizer(tokenizer=tokenize_data)),
            ('tfidf', TfidfTransformer(use_idf=True)),
            ('classifier', GradientBoostingClassifier(random_state=0, max_depth=3, learning_rate=0.1))
            ])

In [ ]:
%time model_gdboost = pipeline_gdboost.fit(train_X, train_y)

CPU times: user 30.2 s, sys: 31 ms, total: 30.3 s
Wall time: 30.3 s


In [ ]:
prediction_gdboost=model_gdboost.predict(test_X)

### 그라디언트 부스트 모델 결과

In [ ]:
print (classification_report(test_y, prediction_gdboost))

              precision    recall  f1-score   support

           0       0.45      0.63      0.52      1240
           1       0.10      0.06      0.08       309
           2       0.00      0.00      0.00        51
           3       0.19      0.15      0.17       521
           4       0.11      0.09      0.10       350
           5       0.03      0.01      0.02        93
           6       0.03      0.01      0.01       141
           7       0.02      0.02      0.02        59

    accuracy                           0.33      2764
   macro avg       0.12      0.12      0.11      2764
weighted avg       0.27      0.33      0.29      2764



In [ ]:
pipeline_logistic=Pipeline([
            ('bow', CountVectorizer(tokenizer=tokenize_data)),
            ('tfidf', TfidfTransformer(use_idf=False)),
            ('classifier', LogisticRegression(solver='newton-cg'))
            ])

In [ ]:
%time model_logistic = pipeline_logistic.fit(train_X, train_y)

CPU times: user 4.56 s, sys: 5.98 s, total: 10.5 s
Wall time: 3.82 s


In [ ]:
prediction_logistic=model_logistic.predict(test_X)

### 로지스틱 리그레션 모델 결과

In [ ]:
print (classification_report(test_y, prediction_logistic))

              precision    recall  f1-score   support

           0       0.45      0.65      0.53      1240
           1       0.12      0.09      0.10       309
           2       0.00      0.00      0.00        51
           3       0.19      0.13      0.16       521
           4       0.10      0.09      0.10       350
           5       0.00      0.00      0.00        93
           6       0.06      0.01      0.02       141
           7       0.00      0.00      0.00        59

    accuracy                           0.34      2764
   macro avg       0.11      0.12      0.11      2764
weighted avg       0.26      0.34      0.29      2764



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
def one_hot_result (result):
    ret=[]
    for i in result:
        onehot=[0,0,0,0,0,0,0,0]
        onehot[i]=1
        ret.append(onehot)
    return ret

In [ ]:
tot_onehot=np.array(one_hot_result(prediction))+np.array(one_hot_result(prediction_logistic)) +np.array(one_hot_result(prediction_randomforest))+np.array(one_hot_result(prediction_gdboost))

In [ ]:
tot_result=np.argmax(tot_onehot, axis=1)

In [ ]:
tot_result

array([0, 0, 0, ..., 4, 0, 0])

### 모델 VOTING 결과

In [ ]:
print (classification_report(test_y, tot_result))

              precision    recall  f1-score   support

           0       0.45      0.72      0.55      1240
           1       0.11      0.06      0.08       309
           2       0.00      0.00      0.00        51
           3       0.19      0.10      0.14       521
           4       0.11      0.08      0.09       350
           5       0.00      0.00      0.00        93
           6       0.00      0.00      0.00       141
           7       0.00      0.00      0.00        59

    accuracy                           0.36      2764
   macro avg       0.11      0.12      0.11      2764
weighted avg       0.26      0.36      0.29      2764


### 평가 파일 생성

In [ ]:
leader_test= leader_test.utterance

In [ ]:
result1= model_nb.predict(leader_test)
result2= model_logistic.predict(leader_test)
result3= model_randomforest.predict(leader_test)
result4= model_gdboost.predict(leader_test)

tot_result_onehot=np.array(one_hot_result(result1))+np.array(one_hot_result(result2)) +np.array(one_hot_result(result3))+np.array(one_hot_result(result4))

In [ ]:
tot_result_leader=np.argmax(tot_result_onehot, axis=1)

In [ ]:
len(tot_result_leader)

1623

In [ ]:
predict_leader=ind_to_str(tot_result_leader)

In [ ]:
predict_leader

,0
0,neutral
1,surprise
2,neutral
3,neutral
4,non-neutral
...,...
1618,neutral
1619,joy
1620,neutral
1621,neutral


In [ ]:
predict_leader.rename(columns = {'0' : 'Id'}, inplace = True)

In [ ]:
import csv
f = open('/content/drive/MyDrive/sample_eng_nb.csv','a', newline='')
wr = csv.writer(f)
wr.writerow(['Id','Expected'])
for i in range(len(predict_leader)):
  wr.writerow([i,predict_leader[0][i]])
f.close()

### 캐글 점수 0.44882